In [15]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
from scipy.interpolate import interp1d
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [16]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
import bokeh
output_notebook()

Loading BokehJS ...

In [17]:
%run ./_EvaluationUtil.ipynb

In [18]:
nomeModelo = 'publico-COMPLETO.bin'
pathModelo = os.path.join('..', '..', 'resources', 'Modelos', nomeModelo)
model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(pathModelo)

In [19]:
pastaArquivoCorpus   = join('..', '..', 'resources', 'Corpora', '3- Textos Preprocessados', '_Consolidados')
nomeArquivoCorpus = 'publico-COMPLETO.txt'
texto = leTextoDeArquivo(join(pastaArquivoCorpus, nomeArquivoCorpus))

In [20]:
portugues = ['exploracao', 'producao', 'perfuracao', 'sismica', 'campo', 'bacia',  'plataforma', 'oleo', 'hidrocarboneto', 'combustivel', 'duto',
             'rocha', 'falha', 'poco', 'porosidade', 'permeabilidade', 'viscosidade',
             'pressao', 'arenito', 'sal', 'sedimento', 'brasil']
ingles = ['exploration', 'production', 'drilling', 'seismic', 'field', 'basin', 'platform', 'oil', 'hydrocarbon', 'fuel', 'pipe', 'rock', 'fault', 'wells', 'porosity', 'permeability', 'viscosity', 'pressure', 'sandstone', 'salt', 'sediment', 'brazil']

In [21]:
texto.count('exploration exploracao')

0

In [22]:
def getMetricsTable(texto, wordsEn, wordsPt):
    nomesColunas = ['a, b', 'Count a', 'Count b', 'Count (a,b)'] #, 'Count (a,b) in window 10']
    rows = []
    for i in range(len(wordsEn)):
        rows.append(
        [
            '('+wordsEn[i]+', '+wordsPt[i]+')',
            texto.count(wordsEn[i]),
            texto.count(wordsPt[i]),
            texto.count(wordsEn[i]+' '+wordsPt[i]) + texto.count(wordsPt[i]+' '+wordsEn[i]),
            #0
        ])
        
    print(rows)
    
    df = pd.DataFrame(rows, columns=nomesColunas)
    return df
    
    

In [45]:
#df = getMetricsTable(texto, ingles, portugues)
#pretty_print(df)

[['(exploration, exploracao)', 6536, 29145, 5], ['(production, producao)', 47107, 184302, 56], ['(drilling, perfuracao)', 8772, 26133, 11], ['(seismic, sismica)', 10043, 15660, 4], ['(field, campo)', 27093, 105316, 103], ['(basin, bacia)', 28276, 99221, 25], ['(platform, plataforma)', 3905, 32537, 12], ['(oil, oleo)', 121826, 383708, 320], ['(hydrocarbon, hidrocarboneto)', 11394, 38382, 2], ['(fuel, combustivel)', 49442, 41099, 52], ['(pipe, duto)', 20840, 178889, 13], ['(rock, rocha)', 18684, 55964, 8], ['(fault, falha)', 17000, 56189, 30], ['(wells, poco)', 9915, 140654, 4], ['(porosity, porosidade)', 5878, 24359, 1], ['(permeability, permeabilidade)', 6010, 27650, 1], ['(viscosity, viscosidade)', 2487, 29953, 3], ['(pressure, pressao)', 27570, 143410, 61], ['(sandstone, arenito)', 6986, 26532, 6], ['(salt, sal)', 31810, 154416, 96], ['(sediment, sedimento)', 102409, 37278, 6], ['(brazil, brasil)', 22487, 150189, 22]]


,"a, b",Count a,Count b,"Count (a,b)"
0,"(exploration, exploracao)",6536,29145,5
1,"(production, producao)",47107,184302,56
2,"(drilling, perfuracao)",8772,26133,11
3,"(seismic, sismica)",10043,15660,4
4,"(field, campo)",27093,105316,103
5,"(basin, bacia)",28276,99221,25
6,"(platform, plataforma)",3905,32537,12
7,"(oil, oleo)",121826,383708,320
8,"(hydrocarbon, hidrocarboneto)",11394,38382,2
9,"(fuel, combustivel)",49442,41099,52


In [ ]:
## Testes de coocorrencia por deslocamento de janela

In [23]:
from collections import defaultdict
def co_occurrence(sentences, vocab, window_size):
    d = defaultdict(int)
    #vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            if token in vocab:
                #vocab.add(token)  # add to vocab
                next_token = text[i+1 : i+1+window_size]
                for t in next_token:
                    if t in vocab:
                        key = tuple( sorted([t, token]) )
                        d[key] += 1

    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [24]:
from collections import defaultdict
def co_occurrence_new_test(sentences, vocab, window_size):
    d = defaultdict(int)
    #vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            if token in vocab:
                #vocab.add(token)  # add to vocab
                next_token = text[i+1 : i+1+window_size]
                for t in next_token:
                    key = tuple( sorted([token, token]) )
                    d[key] += 1
                    if t in vocab:
                        key = tuple( sorted([t, token]) )
                        d[key] += 1

    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [67]:
#df.at['<number>','xvi']

3

In [72]:
## Dados completos - iterando todo o texto para montagem de matriz de coocorrencia com window size=5

In [25]:
def getMetricsTableFromMatrix_OLD(texto, wordsEn, wordsPt, df):
    nomesColunas = ['a, b', 'Count a', 'Count b', 'Count (a,b)', 'Count (a,b) in window=5', 'Count a in window=5', 'Count b in window=5'] #, 'Count (a,b) in window 10']
    rows = []
    for i in range(len(wordsEn)):
        rows.append(
        [
            '('+wordsEn[i]+', '+wordsPt[i]+')',
            texto.count(wordsEn[i]),
            texto.count(wordsPt[i]),
            texto.count(wordsEn[i]+' '+wordsPt[i]) + texto.count(wordsPt[i]+' '+wordsEn[i]),
            df.at[wordsEn[i],wordsPt[i]],
            df.at[wordsEn[i],wordsEn[i]],
            df.at[wordsPt[i],wordsPt[i]]
            #0
        ])
        
    print(rows)
    
    df = pd.DataFrame(rows, columns=nomesColunas)
    return df
    
    

In [26]:
def getMetricsTableFromMatrix(texto, wordsEn, wordsPt, df):
    nomesColunas = ['a, b', 'Count a', 'Count b', 'Count (a,b)'] #, 'Count (a,b) in window 10']
    rows = []
    for i in range(len(wordsEn)):
        rows.append(
        [
            '('+wordsEn[i]+', '+wordsPt[i]+')',
            df.at[wordsEn[i],wordsEn[i]],
            df.at[wordsPt[i],wordsPt[i]],
            df.at[wordsEn[i],wordsPt[i]]
            #0
        ])
        
    print(rows)
    
    df = pd.DataFrame(rows, columns=nomesColunas)
    return df

In [27]:
#vocabulary = [str(i) for i in model.vocab]

In [ ]:
#df = getMetricsTableFromMatrix(texto, ingles, portugues, co_occurrence_matrix)
#pretty_print(df)

In [29]:
# calculando a matriz de co-ocorrencia, apenas para as palavras utilizadas nas analogias
%timeit
vocabulary = portugues + ingles
text = texto.split("\n")
co_occurrence_matrix = co_occurrence(text, vocabulary, 5)

In [31]:
%timeit
vocabulary = portugues + ingles
text = texto.split("\n")
co_occurrence_matrix = co_occurrence_ORIGINAL(text, vocabulary, 5)


In [13]:
df = getMetricsTableFromMatrix(texto, ingles, portugues, co_occurrence_matrix)
pretty_print(df)

[['(exploration, exploracao)', 6536, 29145, 5, 11], ['(production, producao)', 47107, 184302, 56, 311], ['(drilling, perfuracao)', 8772, 26133, 11, 153], ['(seismic, sismica)', 10043, 15660, 4, 16], ['(field, campo)', 27093, 105316, 103, 78], ['(basin, bacia)', 28276, 99221, 25, 70], ['(platform, plataforma)', 3905, 32537, 12, 25], ['(oil, oleo)', 121826, 383708, 320, 1026], ['(hydrocarbon, hidrocarboneto)', 11394, 38382, 2, 3], ['(fuel, combustivel)', 49442, 41099, 52, 392], ['(pipe, duto)', 20840, 178889, 13, 9], ['(rock, rocha)', 18684, 55964, 8, 33], ['(fault, falha)', 17000, 56189, 30, 53], ['(wells, poco)', 9915, 140654, 4, 2], ['(porosity, porosidade)', 5878, 24359, 1, 17], ['(permeability, permeabilidade)', 6010, 27650, 1, 9], ['(viscosity, viscosidade)', 2487, 29953, 3, 25], ['(pressure, pressao)', 27570, 143410, 61, 296], ['(sandstone, arenito)', 6986, 26532, 6, 18], ['(salt, sal)', 31810, 154416, 96, 82], ['(sediment, sedimento)', 102409, 37278, 6, 7], ['(brazil, brasil)', 2

,"a, b",Count a,Count b,"Count (a,b)","Count (a,b) in window=5"
0,"(exploration, exploracao)",6536,29145,5,11
1,"(production, producao)",47107,184302,56,311
2,"(drilling, perfuracao)",8772,26133,11,153
3,"(seismic, sismica)",10043,15660,4,16
4,"(field, campo)",27093,105316,103,78
5,"(basin, bacia)",28276,99221,25,70
6,"(platform, plataforma)",3905,32537,12,25
7,"(oil, oleo)",121826,383708,320,1026
8,"(hydrocarbon, hidrocarboneto)",11394,38382,2,3
9,"(fuel, combustivel)",49442,41099,52,392


In [15]:
df = getMetricsTableFromMatrix(texto, ingles, portugues, co_occurrence_matrix)
pretty_print(df)

[['(exploration, exploracao)', 6536, 29145, 5, 11, 102, 515], ['(production, producao)', 47107, 184302, 56, 311, 1700, 8131], ['(drilling, perfuracao)', 8772, 26133, 11, 153, 394, 1330], ['(seismic, sismica)', 10043, 15660, 4, 16, 337, 480], ['(field, campo)', 27093, 105316, 103, 78, 540, 2255], ['(basin, bacia)', 28276, 99221, 25, 70, 940, 3372], ['(platform, plataforma)', 3905, 32537, 12, 25, 67, 727], ['(oil, oleo)', 121826, 383708, 320, 1026, 5971, 11787], ['(hydrocarbon, hidrocarboneto)', 11394, 38382, 2, 3, 70, 56], ['(fuel, combustivel)', 49442, 41099, 52, 392, 1184, 1531], ['(pipe, duto)', 20840, 178889, 13, 9, 496, 924], ['(rock, rocha)', 18684, 55964, 8, 33, 319, 1045], ['(fault, falha)', 17000, 56189, 30, 53, 718, 1992], ['(wells, poco)', 9915, 140654, 4, 2, 510, 5060], ['(porosity, porosidade)', 5878, 24359, 1, 17, 284, 1112], ['(permeability, permeabilidade)', 6010, 27650, 1, 9, 333, 1612], ['(viscosity, viscosidade)', 2487, 29953, 3, 25, 139, 1783], ['(pressure, pressao)'

,"a, b",Count a,Count b,"Count (a,b)","Count (a,b) in window=5","Count a,b in window=5",Count b in window=5
0,"(exploration, exploracao)",6536,29145,5,11,102,515
1,"(production, producao)",47107,184302,56,311,1700,8131
2,"(drilling, perfuracao)",8772,26133,11,153,394,1330
3,"(seismic, sismica)",10043,15660,4,16,337,480
4,"(field, campo)",27093,105316,103,78,540,2255
5,"(basin, bacia)",28276,99221,25,70,940,3372
6,"(platform, plataforma)",3905,32537,12,25,67,727
7,"(oil, oleo)",121826,383708,320,1026,5971,11787
8,"(hydrocarbon, hidrocarboneto)",11394,38382,2,3,70,56
9,"(fuel, combustivel)",49442,41099,52,392,1184,1531


In [32]:
df = getMetricsTableFromMatrix_OLD(texto, ingles, portugues, co_occurrence_matrix)
pretty_print(df)


[['(exploration, exploracao)', 6536, 29145, 5, 11, 102, 515], ['(production, producao)', 47107, 184302, 56, 311, 1700, 8131], ['(drilling, perfuracao)', 8772, 26133, 11, 153, 394, 1330], ['(seismic, sismica)', 10043, 15660, 4, 16, 337, 480], ['(field, campo)', 27093, 105316, 103, 78, 540, 2255], ['(basin, bacia)', 28276, 99221, 25, 70, 940, 3372], ['(platform, plataforma)', 3905, 32537, 12, 25, 67, 727], ['(oil, oleo)', 121826, 383708, 320, 1026, 5971, 11787], ['(hydrocarbon, hidrocarboneto)', 11394, 38382, 2, 3, 70, 56], ['(fuel, combustivel)', 49442, 41099, 52, 392, 1184, 1531], ['(pipe, duto)', 20840, 178889, 13, 9, 496, 924], ['(rock, rocha)', 18684, 55964, 8, 33, 319, 1045], ['(fault, falha)', 17000, 56189, 30, 53, 718, 1992], ['(wells, poco)', 9915, 140654, 4, 2, 510, 5060], ['(porosity, porosidade)', 5878, 24359, 1, 17, 284, 1112], ['(permeability, permeabilidade)', 6010, 27650, 1, 9, 333, 1612], ['(viscosity, viscosidade)', 2487, 29953, 3, 25, 139, 1783], ['(pressure, pressao)'

,"a, b",Count a,Count b,"Count (a,b)","Count (a,b) in window=5",Count a in window=5,Count b in window=5
0,"(exploration, exploracao)",6536,29145,5,11,102,515
1,"(production, producao)",47107,184302,56,311,1700,8131
2,"(drilling, perfuracao)",8772,26133,11,153,394,1330
3,"(seismic, sismica)",10043,15660,4,16,337,480
4,"(field, campo)",27093,105316,103,78,540,2255
5,"(basin, bacia)",28276,99221,25,70,940,3372
6,"(platform, plataforma)",3905,32537,12,25,67,727
7,"(oil, oleo)",121826,383708,320,1026,5971,11787
8,"(hydrocarbon, hidrocarboneto)",11394,38382,2,3,70,56
9,"(fuel, combustivel)",49442,41099,52,392,1184,1531
